## Questions
<ul>
    <li><del>What range of years does the provided database cover?</del></li>
<br> 
<li><del>Find the name and height of the shortest player in the database. How many games did he play in? What is the name of the team for which he played?</del></li>
<br> 
<li><del>Find all players in the database who played at Vanderbilt University. Create a list showing each player’s first and last names as well as the total salary they earned in the major leagues. Sort this list in descending order by the total salary earned. Which Vanderbilt player earned the most money in the majors?</del></li>
<br> 
<li><del>Using the fielding table, group players into three groups based on their position: label players with position OF as "Outfield", those with position "SS", "1B", "2B", and "3B" as "Infield", and those with position "P" or "C" as "Battery". Determine the number of putouts made by each of these three groups in 2016.</del></li>
<br> 
<li><del>Find the average number of strikeouts per game by decade since 1920. Round the numbers you report to 2 decimal places. Do the same for home runs per game. Do you see any trends?</del></li>
<br> 
<li><del>Find the player who had the most success stealing bases in 2016, where success is measured as the percentage of stolen base attempts which are successful. (A stolen base attempt results either in a stolen base or being caught stealing.) Consider only players who attempted at least 20 stolen bases.</del></li>
<br> 
<li>From 1970 – 2016, what is the largest number of wins for a team that did not win the world series? What is the smallest number of wins for a team that did win the world series? Doing this will probably result in an unusually small number of wins for a world series champion – determine why this is the case. Then redo your query, excluding the problem year. How often from 1970 – 2016 was it the case that a team with the most wins also won the world series? What percentage of the time?</li>
<br> 
<li>Using the attendance figures from the homegames table, find the teams and parks which had the top 5 average attendance per game in 2016 (where average attendance is defined as total attendance divided by number of games). Only consider parks where there were at least 10 games played. Report the park name, team name, and average attendance. Repeat for the lowest 5 average attendance.</li>
<br> 
<li>Which managers have won the TSN Manager of the Year award in both the National League (NL) and the American League (AL)? Give their full name and the teams that they were managing when they won the award.</li> 
</ul>

## Bonus Questions
<ul>
<li>Analyze all the colleges in the state of Tennessee. Which college has had the most success in the major leagues. Use whatever metric for success you like - number of players, number of games, salaries, world series wins, etc.</li>
<br>
<li>Is there any correlation between number of wins and team salary? Use data from 2000 and later to answer this question. As you do this analysis, keep in mind that salaries across the whole league tend to increase together, so you may want to look on a year-by-year basis.</li>
<br>
<li>It is thought that since left-handed pitchers are more rare, causing batters to face them less often, that they are more effective. Investigate this claim and present evidence to either support or dispute this claim. First, determine just how rare left-handed pitchers are compared with right-handed pitchers. Are left-handed pitchers more likely to win the Cy Young Award? Are they more likely to make it into the hall of fame?</li>

In [1]:
import os

import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
username = os.getenv('POSTGRES_USERNAME')
password = os.getenv('POSTGRES_PASSWORD')


postgres_connection_string = "postgres://{username}:{password}@{host}:{port}/{database}?gssencmode=disable".format(
    username=username,
    password=password,
    host="localhost",
    port="5432",
    database="lahman_baseball"
)

In [4]:
engine = create_engine(postgres_connection_string)

In [5]:
batting_sql = "SELECT * FROM batting;"
teams_sql = "SELECT * FROM teams;"
people_sql = "SELECT * FROM people;"
appearances_sql = "SELECT * FROM appearances;"
collegeplaying_sql = "SELECT * FROM collegeplaying;"
schools_sql = "SELECT * FROM schools;"
salaries_sql = "SELECT * FROM salaries;"
fielding_sql = "SELECT * FROM fielding;"
pitching_sql = "SELECT * FROM pitching;"
seriespost_sql = "SELECT * FROM seriespost;"

In [6]:
batting_df = pd.read_sql(batting_sql, con=engine)
teams_df = pd.read_sql(teams_sql, con=engine)
people_df = pd.read_sql(people_sql, con=engine)
appearances_df = pd.read_sql(appearances_sql, con=engine)
collegeplaying_df = pd.read_sql(collegeplaying_sql, con=engine)
schools_df = pd.read_sql(schools_sql, con=engine)
salaries_df = pd.read_sql(salaries_sql, con=engine)
fielding_df = pd.read_sql(fielding_sql, con=engine)
pitching_df = pd.read_sql(pitching_sql, con=engine)
seriespost_df = pd.read_sql(seriespost_sql, con=engine)

---
#### What range of years does the provided database cover?

In [7]:
teams_df['yearid'].min()

1871

In [8]:
teams_df['yearid'].max()

2016

---
#### Find the name and height of the shortest player in the database. How many games did he play in? What is the name of the team for which he played?

In [9]:
shortest_player = people_df[people_df.height == people_df.height.min()] 

In [10]:
shortest_player['namegiven']

5843    Edward Carl
Name: namegiven, dtype: object

In [11]:
edward_carl_id = list(shortest_player['playerid'])

In [12]:
appearances_df[appearances_df['playerid'].isin(['gaedeed01'])]

,yearid,teamid,lgid,playerid,g_all,gs,g_batting,g_defense,g_p,g_c,...,g_2b,g_3b,g_ss,g_lf,g_cf,g_rf,g_of,g_dh,g_ph,g_pr
35173,1951,SLA,AL,gaedeed01,1,0.0,1,0,0,0,...,0,0,0,0,0,0,0,0.0,1.0,0.0


In [13]:
teams_df['name'][teams_df['teamid'].isin(['SLA'])].head(1)

412    St. Louis Browns
Name: name, dtype: object

---
#### Find all players in the database who played at Vanderbilt University. Create a list showing each player’s first and last names as well as the total salary they earned in the major leagues. Sort this list in descending order by the total salary earned. Which Vanderbilt player earned the most money in the majors?

In [14]:
vandy_id = list(schools_df.loc[schools_df['schoolname'] == 'Vanderbilt University', 'schoolid'])

In [15]:
vandy_players = list(collegeplaying_df.loc[collegeplaying_df['schoolid'] == 'vandy', 'playerid'])

In [16]:
# function to get unique values 
def unique(list1): 
    x = np.array(list1) 
    print(np.unique(x)) 

In [17]:
unique(vandy_players)

['alvarpe01' 'baxtemi01' 'chrisni01' 'colliwi01' 'corajo01' 'embresl01'
 'flahery01' 'grayso01' 'hendrha01' 'katama01' 'lewisje01' 'madissc01'
 'minormi01' 'mooresc01' 'mossma01' 'pauljo01' 'priceda01' 'priorma01'
 'richaan01' 'sandesc01' 'sewelri01' 'sowerje01' 'willimi01' 'zeidjo01']


In [18]:
#Placed a comma between all items returned above.
vandy_players_unique = ['alvarpe01', 'baxtemi01', 'chrisni01', 'colliwi01', 'corajo01', 'embresl01',
 'flahery01', 'grayso01', 'hendrha01', 'katama01', 'lewisje01', 'madissc01',
 'minormi01', 'mooresc01', 'mossma01', 'pauljo01', 'priceda01', 'priorma01',
 'richaan01', 'sandesc01', 'sewelri01', 'sowerje01', 'willimi01', 'zeidjo01']

In [19]:
vandy_names = people_df.query('playerid in @vandy_players_unique')

In [20]:
vandy_names_fl = vandy_names[['playerid', 'namefirst', 'namelast']]

In [21]:
vandy_salaries = salaries_df.query('playerid in @vandy_players_unique')

In [22]:
vandy_total_salaries = vandy_salaries.groupby('playerid')['salary'].sum().to_frame()

In [23]:
pd.merge(vandy_total_salaries, vandy_names_fl, on = 'playerid').sort_values(by = 'salary', ascending = False).head(1)

,playerid,salary,namefirst,namelast
11,priceda01,81851296.0,David,Price


---

#### Using the fielding table, group players into three groups based on their position: label players with position OF as "Outfield", those with position "SS", "1B", "2B", and "3B" as "Infield", and those with position "P" or "C" as "Battery". Determine the number of putouts made by each of these three groups in 2016.

In [24]:
fielding_df['pos_group'] = fielding_df['pos']

In [25]:
fielding_df.loc[fielding_df['pos'] == '1B', 'pos_group'] = 'Infield'
fielding_df.loc[fielding_df['pos'] == '2B', 'pos_group'] = 'Infield'
fielding_df.loc[fielding_df['pos'] == 'SS', 'pos_group'] = 'Infield'
fielding_df.loc[fielding_df['pos'] == '3B', 'pos_group'] = 'Infield'
fielding_df.loc[fielding_df['pos'] == 'OF', 'pos_group'] = 'Outfield'
fielding_df.loc[fielding_df['pos'] == 'P', 'pos_group'] = 'Battery'
fielding_df.loc[fielding_df['pos'] == 'C', 'pos_group'] = 'Battery'

In [26]:
fielding_df.groupby('pos_group')['po'].sum().to_frame()

,po
pos_group,
Battery,2575499
Infield,6101378
Outfield,2731506


---
#### Find the average number of strikeouts per game by decade since 1920. Round the numbers you report to 2 decimal places. Do the same for home runs per game. Do you see any trends?

In [27]:
so_by_decade = pitching_df[pitching_df['yearid'] >= 1920]

In [28]:
so_by_decade = so_by_decade.drop(columns = ['playerid', 'stint', 'teamid', 'lgid', 'w', 'l', 'g', 'gs',
       'cg', 'sho', 'sv', 'ipouts', 'h', 'er', 'hr', 'bb', 'baopp',
       'era', 'ibb', 'wp', 'hbp', 'bk', 'bfp', 'gf', 'r', 'sh', 'sf', 'gidp'])

In [29]:
#Python is written in C, and native python functions are inherently slower than C, so using for loops is not ideal. 
#Use .apply() instead! (see below)

#decade_list = []

#for year in so_by_decade.yearid:
    #print(year)
    #decade = year//10*10
    #decade_list.append(decade)

In [30]:
#so_by_decade['decade'] = decade_list

In [31]:
#We can define a function...

def calculate_decade(yearid):
    return yearid//10*10

In [32]:
#...and use .apply()!!!

so_by_decade['decade'] = so_by_decade.yearid.apply(calculate_decade)

In [33]:
so_by_decade.groupby('decade')['so'].mean().to_frame().round(2)


,so
decade,
1920,32.07
1930,37.52
1940,35.83
1950,41.81
1960,56.26
1970,53.64
1980,50.64
1990,47.33
2000,46.57


In [34]:
#For plain math functions, you can do it this way, but once the params become more complicated it will not work.

#so_by_decade['decade'] = so_by_decade.yearid//10*10

In [35]:
hr_by_decade = batting_df[batting_df['yearid'] >= 1920]

In [36]:
hr_by_decade = hr_by_decade.drop(columns = ['playerid', 'stint', 'teamid', 'lgid', 'g', 'ab', 'r', 'h',
       'h2b', 'h3b', 'rbi', 'sb', 'cs', 'bb', 'so', 'ibb', 'hbp', 'sh',
       'sf', 'gidp'])

In [37]:
hr_by_decade['decade'] = hr_by_decade.yearid.apply(calculate_decade)

In [38]:
hr_by_decade.groupby('decade')['hr'].mean().to_frame().round(2)

,hr
decade,
1920,1.86
1930,2.57
1940,2.24
1950,3.39
1960,3.46
1970,3.21
1980,3.28
1990,3.49
2000,3.83


#### Find the player who had the most success stealing bases in 2016, where success is measured as the percentage of stolen base attempts which are successful. (A stolen base attempt results either in a stolen base or being caught stealing.) Consider only players who attempted at least 20 stolen bases.

In [39]:
stolen_bases = batting_df.drop(columns = ['stint', 'teamid', 'lgid', 'g', 'ab', 'r', 'h',
       'h2b', 'h3b', 'hr', 'rbi', 'bb', 'so', 'ibb', 'hbp', 'sh',
       'sf', 'gidp'])

In [40]:
stolen_bases = stolen_bases[stolen_bases ['yearid'] == 2016]
stolen_bases = stolen_bases[stolen_bases ['sb'] >= 20]

In [41]:
stolen_bases['success'] = (stolen_bases.sb / (stolen_bases.cs + stolen_bases.sb) * 100).round(2)

In [42]:
player_sb = stolen_bases.sort_values(by = 'success', ascending = False).head(1)

In [43]:
best_stealer = pd.merge(player_sb, people_df, on = 'playerid', how = 'left')

In [44]:
best_stealer.drop(columns = ['birthyear', 'birthmonth',
       'birthday', 'birthcountry', 'birthstate', 'birthcity', 'deathyear',
       'deathmonth', 'deathday', 'deathcountry', 'deathstate', 'deathcity',
        'weight', 'height', 'bats',
       'throws', 'debut', 'finalgame', 'retroid', 'bbrefid'])

,playerid,yearid,sb,cs,success,namefirst,namelast,namegiven
0,owingch01,2016,21.0,2.0,91.3,Chris,Owings,Christopher Scott


#### From 1970 – 2016, what is the largest number of wins for a team that did not win the world series? What is the smallest number of wins for a team that did win the world series? Doing this will probably result in an unusually small number of wins for a world series champion – determine why this is the case. Then redo your query, excluding the problem year. How often from 1970 – 2016 was it the case that a team with the most wins also won the world series? What percentage of the time?

In [215]:
ws_records_1970_2016 = seriespost_df[seriespost_df['yearid'] >= 1970]
ws_records_1970_2016 = ws_records_1970_2016[ws_records_1970_2016['round'] == 'WS']
ws_records_1970_2016 = ws_records_1970_2016.drop(columns = ['wins', 'losses', 'ties'])

In [216]:
teams_1970_2016 = teams_df[teams_df['yearid'] >= 1970]
teams_1970_2016 = teams_1970_2016[teams_1970_2016['lgwin'] == 'Y']

In [217]:
teams_1970_2016.columns

Index(['yearid', 'lgid', 'teamid', 'franchid', 'divid', 'rank', 'g', 'ghome',
       'w', 'l', 'divwin', 'wcwin', 'lgwin', 'wswin', 'r', 'ab', 'h', 'h2b',
       'h3b', 'hr', 'bb', 'so', 'sb', 'cs', 'hbp', 'sf', 'ra', 'er', 'era',
       'cg', 'sho', 'sv', 'ipouts', 'ha', 'hra', 'bba', 'soa', 'e', 'dp', 'fp',
       'name', 'park', 'attendance', 'bpf', 'ppf', 'teamidbr',
       'teamidlahman45', 'teamidretro'],
      dtype='object')

In [218]:
teams_1970_2016 = teams_1970_2016.drop(columns = ['lgid', 'franchid', 'divid', 'rank', 'g', 'ghome',
       'l', 'divwin', 'wcwin', 'lgwin', 'r', 'ab', 'h', 'h2b',
       'h3b', 'hr', 'bb', 'so', 'sb', 'cs', 'hbp', 'sf', 'ra', 'er', 'era',
       'cg', 'sho', 'sv', 'ipouts', 'ha', 'hra', 'bba', 'soa', 'e', 'dp', 'fp',
       'name', 'park', 'attendance', 'bpf', 'ppf', 'teamidbr',
       'teamidlahman45', 'teamidretro'])

In [219]:
ws_winners_losers = pd.merge(ws_records_1970_2016, teams_1970_2016, left_on = 'yearid', right_on = 'yearid')

In [220]:
ws_winners = ws_winners_losers[ws_winners_losers['wswin'] == 'Y'].reset_index(drop = True)

In [221]:
ws_losers = ws_winners_losers[ws_winners_losers['wswin'] == 'N'].reset_index(drop = True)

In [222]:
team_names_id = teams_df[teams_df['yearid'] >= 1970]
team_names_id = team_names_id[['teamid', 'name']]

In [223]:
ws_winners.sort_values(by = 'w', ascending = True).head()

,yearid,round,teamidwinner,lgidwinner,teamidloser,lgidloser,teamid,w,wswin
11,1981,WS,LAN,NL,NYA,AL,LAN,63,Y
35,2006,WS,SLN,NL,DET,AL,SLN,83,Y
17,1987,WS,MIN,AL,SLN,NL,MIN,85,Y
29,2000,WS,NYA,AL,NYN,NL,NYA,87,Y
43,2014,WS,SFN,NL,KCA,AL,SFN,88,Y


In [224]:
team_names_id[team_names_id['teamid'] == 'LAN'].head(1)

,teamid,name
1552,LAN,Los Angeles Dodgers


In [225]:
ws_losers.sort_values(by = 'w', ascending = False).head()

,yearid,round,teamidwinner,lgidwinner,teamidloser,lgidloser,teamid,w,wswin
33,2004,WS,BOS,AL,SLN,NL,SLN,105,N
18,1988,WS,LAN,NL,OAK,AL,OAK,104,N
28,1999,WS,NYA,AL,ATL,NL,ATL,103,N
20,1990,WS,CIN,NL,OAK,AL,OAK,103,N
0,1970,WS,BAL,AL,CIN,NL,CIN,102,N


In [226]:
team_names_id[team_names_id['teamid'] == 'SLN'].head(1)

,teamid,name
1563,SLN,St. Louis Cardinals


In [227]:
ws_winners.sort_values(by = 'w', ascending = True).head()

,yearid,round,teamidwinner,lgidwinner,teamidloser,lgidloser,teamid,w,wswin
11,1981,WS,LAN,NL,NYA,AL,LAN,63,Y
35,2006,WS,SLN,NL,DET,AL,SLN,83,Y
17,1987,WS,MIN,AL,SLN,NL,MIN,85,Y
29,2000,WS,NYA,AL,NYN,NL,NYA,87,Y
43,2014,WS,SFN,NL,KCA,AL,SFN,88,Y


In [255]:
ws_winners_no1981 = ws_winners.drop(index = 11)

In [256]:
ws_losers_no1981 = ws_losers.drop(index = 11)

In [257]:
ws_winners_no1981 = ws_winners_no1981.rename(columns = {'w' : 'ws_champ_w'})

In [258]:
ws_losers_no1981 = ws_losers_no1981.rename(columns = {'w' : 'ws_loser_w'})

In [259]:
ws_winners_no1981['ws_loser_w'] = ws_losers_no1981['ws_loser_w']

In [260]:
ws_winners_no1981.columns

Index(['yearid', 'round', 'teamidwinner', 'lgidwinner', 'teamidloser',
       'lgidloser', 'teamid', 'ws_champ_w', 'wswin', 'ws_loser_w'],
      dtype='object')

In [261]:
ws_winners_no1981 = ws_winners_no1981.drop(columns = ['yearid', 'round', 'teamidwinner', 'lgidwinner', 'teamidloser',
       'lgidloser', 'teamid', 'wswin'])

In [262]:
ws_winners_no1981.head()

,ws_champ_w,ws_loser_w
0,108,102
1,97,101
2,93,95
3,94,82
4,90,102


In [177]:
#engine.dispose()